# InstaCart

### Overview

* Import Python Libraries
* Establish Secure Connection
* Ingest Data from CSV files
  * Upload data to Snowflake Stages
  * Load data from Snowflake Stages into Snowpark DataFrames
  * Write Data from Snowpark DataFrames into Snowflake Tables
* Perform Exploratory Data Anaysis using Snowpark DataFrames
* Write Transformed Data in Snowflake Table
* Visualize Data using Pandas DataFrames


### Import Python Libraries

In [8]:
# Snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, count, avg, sum, col,lit,listagg,call_builtin,when,count_distinct,array_agg,array_construct,call_udf,sproc,lower
from snowflake.snowpark.version import VERSION

# Misc
import pandas as pd
import json
import altair as alt
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

# For AWS Secrets Manager
import boto3
from botocore.exceptions import ClientError

### Establish Secure Connection to Snowflake using AWS Secrets Manager

*NOTE: Other options include loading credentials from a file (for example, connection.json), Okta, SSO, MFA*

In [4]:
# Load Snowflake connection details from AWS Secrets Manager
def get_aws_sf_connection_details(secret_name,region_name):
    
    # Create a Secrets Manager boto3 client
    boto3_session = boto3.session.Session()
    client = boto3_session.client(service_name='secretsmanager',region_name=region_name)
    
    get_secret_value_response = None

    try:
        # Get secret values(s) based on the passed in secret name
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)['SecretString']
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            raise e

    return get_secret_value_response
        
# Create Snowflake Session object
connection_parameters = json.loads(get_aws_sf_connection_details('dash-sfdevrel-connection','us-west-2'))
# connection_parameters['authenticator'] = 'username_password_mfa'
session = Session.builder.configs(connection_parameters).create()

snowflake_environment = session.sql('select current_role(), current_warehouse(), current_database(), current_schema(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('Role                        : {}'.format(snowflake_environment[0][0]))
print('Warehouse                   : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

Role                        : ACCOUNTADMIN
Warehouse                   : DASH_S
Database                    : DASH_DB
Schema                      : DASH_SCHEMA
Snowflake version           : 7.8.1
Snowpark for Python version : 1.0.0


### Ingest Data

#### Upload data in CSV fies onto Snowflake Stage

*NOTE: This is a one-time operation that can also be performed using SnowSQL.*

In [5]:
# session.file.put("../Data/instacart-market-basket-analysis/order_products__prior.csv",stage_location="@dash_files",overwrite=True,auto_compress=False)
# session.file.put("../Data/instacart-market-basket-analysis/aisles.csv",stage_location="@dash_files",overwrite=True,auto_compress=False)
# session.file.put("../Data/instacart-market-basket-analysis/departments.csv",stage_location="@dash_files",overwrite=True,auto_compress=False)
# session.file.put("../Data/instacart-market-basket-analysis/products.csv",stage_location="@dash_files",overwrite=True,auto_compress=False)
# session.file.put("../Data/instacart-market-basket-analysis/orders.csv",stage_location="@dash_files",overwrite=True,auto_compress=False)
# session.file.put("../Data/instacart-market-basket-analysis/order_products__train.csv",stage_location="@dash_files",overwrite=True,auto_compress=False)

[PutResult(source='order_products__prior.csv', target='order_products__prior.csv', source_size=577550706, target_size=577550720, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

#### Read data from Snowflake Stages and load it into Snowpark DataFrames

##### Aisles

In [4]:
aisles_schema = StructType([StructField("aisle_id", IntegerType()), StructField("aisle", StringType())])
aisles = session.read.options({"skip_header": 1}).schema(aisles_schema).csv("@dash_files/aisles.csv")
aisles.show()

-------------------------------------------
|"AISLE_ID"  |"AISLE"                     |
-------------------------------------------
|1           |prepared soups salads       |
|2           |specialty cheeses           |
|3           |energy granola bars         |
|4           |instant foods               |
|5           |marinades meat preparation  |
|6           |other                       |
|7           |packaged meat               |
|8           |bakery desserts             |
|9           |pasta sauce                 |
|10          |kitchen supplies            |
-------------------------------------------



##### Departments

In [6]:
departments_schema = StructType([StructField("department_id", IntegerType()), StructField("department", StringType())])
departments = session.read.options({"skip_header": 1}).schema(departments_schema).csv("@dash_files/departments.csv")
departments.show()

-------------------------------------
|"DEPARTMENT_ID"  |"DEPARTMENT"     |
-------------------------------------
|1                |frozen           |
|2                |other            |
|3                |bakery           |
|4                |produce          |
|5                |alcohol          |
|6                |international    |
|7                |beverages        |
|8                |pets             |
|9                |dry goods pasta  |
|10               |bulk             |
-------------------------------------



##### Order Products (Historical set)

In [7]:
# order_id,product_id,add_to_cart_order,reordered
order_products_prior_schema = StructType([StructField("order_id", IntegerType()), StructField("product_id", IntegerType()),StructField("add_to_cart_order", IntegerType()), StructField("reordered", IntegerType())])
order_products_prior = session.read.options({"skip_header": 1}).schema(order_products_prior_schema).csv("@dash_files/order_products__prior.csv")
order_products_prior.show()

-----------------------------------------------------------------
|"ORDER_ID"  |"PRODUCT_ID"  |"ADD_TO_CART_ORDER"  |"REORDERED"  |
-----------------------------------------------------------------
|2           |33120         |1                    |1            |
|2           |28985         |2                    |1            |
|2           |9327          |3                    |0            |
|2           |45918         |4                    |1            |
|2           |30035         |5                    |0            |
|2           |17794         |6                    |1            |
|2           |40141         |7                    |1            |
|2           |1819          |8                    |1            |
|2           |43668         |9                    |0            |
|3           |33754         |1                    |1            |
-----------------------------------------------------------------



##### Order Products (Training set)

In [8]:
# order_id,product_id,add_to_cart_order,reordered
order_products_train_schema = StructType([StructField("order_id", IntegerType()), StructField("product_id", IntegerType()),StructField("add_to_cart_order", IntegerType()), StructField("reordered", IntegerType())])
order_products_train = session.read.options({"skip_header": 1}).schema(order_products_train_schema).csv("@dash_files/order_products__train.csv")
order_products_train.show()

-----------------------------------------------------------------
|"ORDER_ID"  |"PRODUCT_ID"  |"ADD_TO_CART_ORDER"  |"REORDERED"  |
-----------------------------------------------------------------
|1           |49302         |1                    |1            |
|1           |11109         |2                    |1            |
|1           |10246         |3                    |0            |
|1           |49683         |4                    |0            |
|1           |43633         |5                    |1            |
|1           |13176         |6                    |0            |
|1           |47209         |7                    |0            |
|1           |22035         |8                    |1            |
|36          |39612         |1                    |0            |
|36          |19660         |2                    |1            |
-----------------------------------------------------------------



##### Orders

In [9]:
# order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
# 2539329,1,prior,1,2,08,
# 2398795,1,train,2,3,07,15.0
# 473747,1,prior,3,3,12,21.0
# 2254736,1,test,4,4,07,29.0

orders_schema = StructType(
    [StructField("order_id", IntegerType()), 
    StructField("user_id", IntegerType()),
    StructField("eval_set", StringType()),
    StructField("order_number", IntegerType()),
    StructField("order_dow", IntegerType()),
    StructField("order_hour_of_day", StringType()),
    StructField("days_since_prior_order", FloatType())
    ])
orders = session.read.options({"skip_header": 1}).schema(orders_schema).csv("@dash_files/orders.csv")
orders.show()

-----------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"USER_ID"  |"EVAL_SET"  |"ORDER_NUMBER"  |"ORDER_DOW"  |"ORDER_HOUR_OF_DAY"  |"DAYS_SINCE_PRIOR_ORDER"  |
-----------------------------------------------------------------------------------------------------------------------
|2539329     |1          |prior       |1               |2            |08                   |NULL                      |
|2398795     |1          |prior       |2               |3            |07                   |15.0                      |
|473747      |1          |prior       |3               |3            |12                   |21.0                      |
|2254736     |1          |prior       |4               |4            |07                   |29.0                      |
|431534      |1          |prior       |5               |4            |15                   |28.0                      |
|3367565     |1          |prior       |6

##### Products

*TIP: The use of FIELD_OPTIONALLY_ENCLOSED_BY parameter below makes ingesting Product records where the name might include double-quotes really easy without having to add an extra preprocessing step to handle / remove the double-quotes.*

In [10]:
# product_id,product_name,aisle_id,department_id
# 1,Chocolate Sandwich Cookies,61,19
# 2,All-Seasons Salt,104,13
# 3,Robust Golden Unsweetened Oolong Tea,94,7
# 4,"Smart Ones Classic Favorites, Mini Rigatoni With Vodka Cream Sauce",38,1
# 5,Green Chile Anytime Sauce,5,13

products_schema = StructType(
    [StructField("product_id", IntegerType()), 
    StructField("product_name", StringType()),
    StructField("aisle_id", IntegerType()),
    StructField("department_id", IntegerType())
    ])
products = session.read.options({"skip_header": 1, "FIELD_OPTIONALLY_ENCLOSED_BY": '"'}).schema(products_schema).csv("@dash_files/products.csv")
products.show()

----------------------------------------------------------------------------------------------------
|"PRODUCT_ID"  |"PRODUCT_NAME"                                      |"AISLE_ID"  |"DEPARTMENT_ID"  |
----------------------------------------------------------------------------------------------------
|1             |Chocolate Sandwich Cookies                          |61          |19               |
|2             |All-Seasons Salt                                    |104         |13               |
|3             |Robust Golden Unsweetened Oolong Tea                |94          |7                |
|4             |Smart Ones Classic Favorites Mini Rigatoni With...  |38          |1                |
|5             |Green Chile Anytime Sauce                           |5           |13               |
|6             |Dry Nose Oil                                        |11          |11               |
|7             |Pure Coconut Water With Orange                      |98          |7        

#### Write Data from Snowpark DataFrames into Snowflake tables

*NOTE: Writing data is a one-time operation. Once the data is loaded into Snowflake tables, it can be easily loaded into Snowpark DataFrames as shown below in the next section.*

In [ ]:
# aisles.copy_into_table('insta_aisles') 
# departments.copy_into_table('insta_departments')
# order_products_prior.copy_into_table('insta_order_products_prior')
# order_products_train.copy_into_table('insta_order_products_train')
# orders.copy_into_table('insta_orders')
# products.copy_into_table('insta_products')

### Exploratory Data Analysis using Snowpark DataFrames

#### Load data from Snowflake tables into Snowpark DataFrames

In [5]:
aisles = session.table('insta_aisles')
departments = session.table('insta_departments')
order_products_prior = session.table('insta_order_products_prior')
order_products_train = session.table('insta_order_products_train')
orders = session.table('insta_orders')
products = session.table('insta_products')

#### Busiest day of the week

In [4]:
sf_busiest_dow = orders.group_by('order_dow') \
        .agg(count('order_id').as_('total_orders')) \
        .sort('total_orders',ascending=False) \
        .select(when(col('order_dow') == 0,'Sunday').
                when(col('order_dow') == 1,'Monday').
                when(col('order_dow') == 2,'Tuesday').
                when(col('order_dow') == 3,'Wednesday').
                when(col('order_dow') == 4,'Thursday').
                when(col('order_dow') == 5,'Friday').
                otherwise('Saturday').as_('DOW'),
                'total_orders')
sf_busiest_dow.show()

------------------------------
|"DOW"      |"TOTAL_ORDERS"  |
------------------------------
|Sunday     |600905          |
|Monday     |587478          |
|Tuesday    |467260          |
|Friday     |453368          |
|Saturday   |448761          |
|Wednesday  |436972          |
|Thursday   |426339          |
------------------------------



#### Breakdown of Orders by Hour of the Day

In [5]:
sf_orders_by_hour = orders.group_by('order_hour_of_day') \
        .agg(count('order_id').as_('total_orders')) \
        .sort('order_hour_of_day',ascending=True)
sf_orders_by_hour.show(24)

----------------------------------------
|"ORDER_HOUR_OF_DAY"  |"TOTAL_ORDERS"  |
----------------------------------------
|00                   |22758           |
|01                   |12398           |
|02                   |7539            |
|03                   |5474            |
|04                   |5527            |
|05                   |9569            |
|06                   |30529           |
|07                   |91868           |
|08                   |178201          |
|09                   |257812          |
|10                   |288418          |
|11                   |284728          |
|12                   |272841          |
|13                   |277999          |
|14                   |283042          |
|15                   |283639          |
|16                   |272553          |
|17                   |228795          |
|18                   |182912          |
|19                   |140569          |
|20                   |104292          |
|21             

#### Top 10 Popular Items

In [6]:
sf_top_10_items = order_products_prior.join(products,['product_id']) \
    .group_by('product_name') \
    .agg(count('order_id').as_('total_orders')) \
    .sort('total_orders',ascending=False).limit(10)
sf_top_10_items.show()

-------------------------------------------
|"PRODUCT_NAME"          |"TOTAL_ORDERS"  |
-------------------------------------------
|Banana                  |472565          |
|Bag of Organic Bananas  |379450          |
|Organic Strawberries    |264683          |
|Organic Baby Spinach    |241921          |
|Organic Hass Avocado    |213584          |
|Organic Avocado         |176815          |
|Large Lemon             |152657          |
|Strawberries            |142951          |
|Limes                   |140627          |
|Organic Whole Milk      |137905          |
-------------------------------------------



#### Shelf Space by Department

In [16]:
sf_shelf_space_by_dept = products.join(departments,['department_id'],join_type='inner') \
    .group_by('department') \
    .agg(count_distinct('product_id').as_('total_distinct_products')) \
    .sort('total_distinct_products',ascending=False).limit(10)
sf_shelf_space_by_dept.show()

-----------------------------------------------
|"DEPARTMENT"     |"TOTAL_DISTINCT_PRODUCTS"  |
-----------------------------------------------
|personal care    |6563                       |
|snacks           |6264                       |
|pantry           |5371                       |
|beverages        |4365                       |
|frozen           |4007                       |
|dairy eggs       |3449                       |
|household        |3085                       |
|canned goods     |2092                       |
|dry goods pasta  |1858                       |
|produce          |1684                       |
-----------------------------------------------



#### Prepare Data for Frequent Pattern Matching

In [7]:
sf_basket = products.join(order_products_train,['product_id'],join_type='inner') \
    .select('product_name','order_id') \
    .group_by('order_id') \
    .agg(array_agg('product_name').as_('items'))
sf_basket.show()

-------------------------------------------------------------------
|"ORDER_ID"  |"ITEMS"                                             |
-------------------------------------------------------------------
|2415        |[                                                   |
|            |  "Organic Strawberries",                           |
|            |  "Banana",                                         |
|            |  "Small Hass Avocado",                             |
|            |  "Asparagus",                                      |
|            |  "Honey Nut Cheerios",                             |
|            |  "Luden's Throat Drops Wild Cherry"                |
|            |]                                                   |
|2191        |[                                                   |
|            |  "Organic Baby Carrots",                           |
|            |  "Gluten Free White Sandwich Bread",               |
|            |  "Organic Spring Mix",           

In [9]:
@udf(session=session,name='to_lowercase',replace=True,is_permanent=True,stage_location='@dash_udfs')
def to_lowercase(txt: str) -> str:
    return txt.lower()

In [13]:
products.select("PRODUCT_NAME",call_udf('to_lowercase',col('PRODUCT_NAME')).as_('PRODUCT_NAME_LOWERCASE')).show()

-----------------------------------------------------------------------------------------------------------
|"PRODUCT_NAME"                                      |"PRODUCT_NAME_LOWERCASE"                            |
-----------------------------------------------------------------------------------------------------------
|Chocolate Sandwich Cookies                          |chocolate sandwich cookies                          |
|All-Seasons Salt                                    |all-seasons salt                                    |
|Robust Golden Unsweetened Oolong Tea                |robust golden unsweetened oolong tea                |
|Smart Ones Classic Favorites Mini Rigatoni With...  |smart ones classic favorites mini rigatoni with...  |
|Green Chile Anytime Sauce                           |green chile anytime sauce                           |
|Dry Nose Oil                                        |dry nose oil                                        |
|Pure Coconut Water With Ora

### Write Transformed Data into Snowflake Table

In [31]:
sf_basket.write.mode('overwrite').save_as_table('insta_basket')

### Visualize Data using Pandas DataFrames

*NOTE: Data can be easily loaded from Snowpark DataFrames into Pandas DataFrames for visualizing the data.*

In [8]:
pd_busiest_dow = sf_busiest_dow.to_pandas()
alt.Chart(pd_busiest_dow, title="Busiest Day Of The Week").mark_bar(color="#29b5e8").encode(
    x = alt.X("DOW:O",sort=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']),
    y = "TOTAL_ORDERS:Q"
).properties(
    width=1550,
    height=500
).configure_title(fontSize=24)

alt.Chart(...)

In [9]:
pd_top_10_items = sf_top_10_items.to_pandas()
alt.Chart(pd_top_10_items, title="Top 10 Popular Items").mark_bar(color="#29b5e8").encode(
    x = alt.X("PRODUCT_NAME:O",sort=None),
    y = "TOTAL_ORDERS:Q"
).properties(
    width=1550,
    height=500
).configure_title(fontSize=24)

alt.Chart(...)

In [11]:
pd_orders_by_hour = sf_orders_by_hour.to_pandas()
alt.Chart(pd_orders_by_hour, title="Orders By Hour").mark_line(color="#29b5e8").encode(
    x = alt.X("ORDER_HOUR_OF_DAY",sort=None),
    y = "TOTAL_ORDERS:Q"
).properties(
    width=1550,
    height=500
).configure_title(fontSize=24)

alt.Chart(...)

In [57]:
pd_shelf_space_by_dept = sf_shelf_space_by_dept.to_pandas()
base = alt.Chart(pd_shelf_space_by_dept, title="Top 10 Shelf Space By Department").mark_arc(outerRadius=200).encode(
    theta=alt.Theta("TOTAL_DISTINCT_PRODUCTS:Q", stack=True), color=alt.Color("DEPARTMENT:N")
).properties(
    width=1550,
    height=500
)
text = base.mark_text(radius=255, size=15).encode(text="DEPARTMENT:N")
(base + text).configure_title(fontSize=24)

alt.LayerChart(...)